In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


In [2]:
!pip install gensim


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [4]:
from gensim.models import Word2Vec # to Load the saved model
from gensim.models.fasttext import load_facebook_model
from tabulate import tabulate
import random
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.fasttext import FastText

In [5]:
en_stop = set(stopwords.words('english'))

In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet', "/kaggle/working/nltk_data/")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...


True

In [7]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
! gunzip "cc.en.300.bin.gz"

--2024-04-20 22:44:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.96, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: 'cc.en.300.bin.gz'

cc.en.300.bin.gz    100%[===================>]   4.19G  92.0MB/s    in 21s     

2024-04-20 22:45:06 (204 MB/s) - 'cc.en.300.bin.gz' saved [4503593528/4503593528]



In [8]:
from gensim.models.fasttext import load_facebook_model
pretrained_fastText_en = load_facebook_model('/kaggle/working/cc.en.300.bin')


In [9]:
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
   creating: /kaggle/working/nltk_data/corpora/wordnet/
  inflating: /kaggle/working/nltk_data/corpora/wordnet/lexnames  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adv  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/ci

In [10]:
import string
import random
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models.fasttext import load_facebook_model

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers
    text = ''.join([i for i in text if not i.isdigit()])

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Remove stopwords and words with less than 3 characters
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) >= 3]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

def get_similar_words(word, topn=10):
    similar_words = pretrained_fastText_en.wv.most_similar(word, topn=topn)
    return similar_words

def get_opposite_words(word, topn=10):
    opposite_words = pretrained_fastText_en.wv.most_similar(negative=[word], topn=topn)
    return opposite_words



In [11]:
# Read the data
yelp_tips = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json", lines=True)

# Process only the first 1000 samples
yelp_tips_subset = yelp_tips[:6000]

# Apply preprocessing to the text data
yelp_tips_subset['preprocessed_text'] = yelp_tips_subset['text'].apply(preprocess_text)

# Select a random word from the preprocessed text
random_sample_index = random.randint(0, len(yelp_tips_subset) - 1)
random_text = yelp_tips_subset.iloc[random_sample_index]['preprocessed_text']
random_word = random.choice(random_text)

# Get top 10 similar words
similar_words = get_similar_words(random_word)

# Get top 10 opposite words
opposite_words = get_opposite_words(random_word)

# Save results to a file
output_file = "word_similarity_results.txt"
with open(output_file, "w") as file:
    file.write(f"Top 10 similar words to '{random_word}': {similar_words}\n")
    file.write(f"Top 10 opposite words to '{random_word}': {opposite_words}\n")

print(f"Results saved to {output_file}")

/tmp/ipykernel_33/1452946384.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_tips_subset['preprocessed_text'] = yelp_tips_subset['text'].apply(preprocess_text)


Results saved to word_similarity_results.txt


In [12]:
from gensim.models.fasttext import FastText
model = FastText(sentences=yelp_tips_subset['preprocessed_text'], vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Save the trained model
model.save("fasttext_model")

In [13]:
tips_text = list(yelp_tips['text'])
tipsSamples = tips_text[:1000]
print("\nSamples of the text\n {}".format(tipsSamples[0:10]))


Samples of the text
 ['Avengers time with the ladies.', 'They have lots of good deserts and tasty cuban sandwiches', "It's open even when you think it isn't", 'Very decent fried chicken', 'Appetizers.. platter special for lunch', 'Chili Cup + Single Cheeseburger with onion, pickle, and relish + Vanilla Coca-Cola...so far.', "Saturday, Dec 7th 2013, ride Patco's Silver Sleigh w/ Santa & his elves on a decorated train into Center City. Trains leave from Lindenwold at 10am, 11:15am, & 12:30pm, and make all stops. Great for kids!", 'This is probably the best place in the cool Springs area to watch a game and eat', 'Tacos', 'Starbucks substitute in boring downtown Tampa. Ugh. Never again!']


In [14]:
import random
from gensim.models import FastText

# Load the trained FastText model
model = FastText.load("fasttext_model")

# Function to select random words from the model's vocabulary
def select_random_words(model, num_words):
    vocabulary = model.wv.index_to_key  # Get the list of words in the vocabulary
    return random.sample(vocabulary, num_words)

# Select random words for testing
random_words = select_random_words(model, 6)

output_file2 = "word_similarity_output.txt"
with open(output_file2, "w") as file:  # Get top 10 similar and opposite words for each random word
    for word in random_words:
        file.write(f"\nWord: {word}\n")

        if word in model.wv.key_to_index:
            # Top 10 similar words
            similar_words = model.wv.most_similar(positive=[word], topn=10)
            file.write("Similar words:\n")
            for similar_word, similarity in similar_words:
                file.write(f"{similar_word}: {similarity:.4f}\n")

            # Top 10 opposite words
            opposite_words = model.wv.most_similar(positive=[word], negative=True, topn=10)
            file.write("\nOpposite words:\n")
            for opposite_word, similarity in opposite_words:
                file.write(f"{opposite_word}: {similarity:.4f}\n")
        else:
            file.write("Word not found in vocabulary.\n")
print(f"Results saved to {output_file2}")

Results saved to word_similarity_output.txt
